# Subway Stations in NYC - Dataset
The dataset contains both stations and complexes. Make sure to avoid double counting a station, if it's already available as a complex. Or, remove *complex* information and only retain station info to cover all stations in NYC.<br><br>
[Reference](https://catalog.data.gov/dataset/mta-subway-stations-and-complexes)
## Orgainsing the dataset

In [1]:
import pandas as pd
stations = pd.read_csv(r"C:\Users\singh\Desktop\TUD (All Semesters)\Courses - Semester 5 (TU Dresden)\Research Task - Spatial Modelling\Code\MTA_Subway_Stations_and_Complexes.csv")
stations.head()

,Complex ID,Is Complex,Number Of Stations In Complex,Stop Name,Display Name,Constituent Station Names,Station IDs,GTFS Stop IDs,Borough,CBD,Daytime Routes,Structure Type,Latitude,Longitude,ADA,ADA Notes
0,1,False,1,Astoria-Ditmars Blvd,Astoria-Ditmars Blvd (N W),Astoria-Ditmars Blvd,1,R01,Q,False,N W,Elevated,40.775036,-73.912034,0,NaN
1,2,False,1,Astoria Blvd,Astoria Blvd (N W),Astoria Blvd,2,R03,Q,False,N W,Elevated,40.770258,-73.917843,1,NaN
2,3,False,1,30 Av,30 Av (N W),30 Av,3,R04,Q,False,N W,Elevated,40.766779,-73.921479,0,NaN
3,4,False,1,Broadway,Broadway (N W),Broadway,4,R05,Q,False,N W,Elevated,40.761820,-73.925508,0,NaN
4,5,False,1,36 Av,36 Av (N W),36 Av,5,R06,Q,False,N W,Elevated,40.756804,-73.929575,0,NaN


In [2]:
# Boroughs available
stations.Borough.unique()

array(['Q', 'M', 'Bk', 'Bx', 'SI'], dtype=object)

In [3]:
# filtering for Q, M and Bx
stations = stations[stations.Borough.isin(["Q","M","Bx"])]
stations.Borough.unique()

array(['Q', 'M', 'Bx'], dtype=object)

In [4]:
# filtering only for stations
stations = stations[stations["Is Complex"] == False]
stations["Is Complex"].unique()

array([False])

In [5]:
# are stations IDs independent? -- Yes
print(len(stations))
print(len(stations["Station IDs"].unique()))

246
246


In [6]:
# filtering out useless columns
stations = stations[['Stop Name', 'Station IDs',
       'Latitude', 'Longitude']]
stations.head()

,Stop Name,Station IDs,Latitude,Longitude
0,Astoria-Ditmars Blvd,1,40.775036,-73.912034
1,Astoria Blvd,2,40.770258,-73.917843
2,30 Av,3,40.766779,-73.921479
3,Broadway,4,40.761820,-73.925508
4,36 Av,5,40.756804,-73.929575


In [7]:
# Creating Point geometries 

import geopandas as gpd
from shapely import Point
stations['coordinates'] = stations.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
stations.head()                                    

,Stop Name,Station IDs,Latitude,Longitude,coordinates
0,Astoria-Ditmars Blvd,1,40.775036,-73.912034,POINT (-73.912034 40.775036)
1,Astoria Blvd,2,40.770258,-73.917843,POINT (-73.917843 40.770258)
2,30 Av,3,40.766779,-73.921479,POINT (-73.921479 40.766779)
3,Broadway,4,40.761820,-73.925508,POINT (-73.925508 40.76182)
4,36 Av,5,40.756804,-73.929575,POINT (-73.929575 40.756804)


In [8]:
# Saved as a Point obj
type(stations.coordinates[0])

shapely.geometry.point.Point

In [9]:
# Creating GeoDataFrame for the df
stations_gdf = gpd.GeoDataFrame(stations, geometry="coordinates", crs=4326)
stations_gdf.head()

,Stop Name,Station IDs,Latitude,Longitude,coordinates
0,Astoria-Ditmars Blvd,1,40.775036,-73.912034,POINT (-73.91203 40.77504)
1,Astoria Blvd,2,40.770258,-73.917843,POINT (-73.91784 40.77026)
2,30 Av,3,40.766779,-73.921479,POINT (-73.92148 40.76678)
3,Broadway,4,40.761820,-73.925508,POINT (-73.92551 40.76182)
4,36 Av,5,40.756804,-73.929575,POINT (-73.92958 40.7568)


## Importing the rentals dataset

In [10]:
# Importing the dataset created from previous notebook
selected_rentals = pd.read_csv(r"C:\Users\singh\Desktop\TUD (All Semesters)\Courses - Semester 5 (TU Dresden)\Research Task - Spatial Modelling\Code\rentals_transformations_and_indicators.csv")
selected_rentals.head()

,#_rentals,datetime,year,month,day,hour,ID,coordinates,#_rentals_lag_1,name_of_day,...,w_avg_prev_day,w_avg_roll_avg,w_avg_lag_2,temp,rhum,prcp,wspd,rush_hr,MapID,coco
0,0,2024-01-01 08:00:00,2024,1,1,8,0,POINT (-73.9383 40.7923272),NaN,Monday,...,0.0,0.0,0.000000,5.0,62.0,0.0,6.0,1,309.0,3.0
1,0,2024-01-01 10:00:00,2024,1,1,10,0,POINT (-73.9383 40.7923272),0.0,Monday,...,0.0,0.0,0.000000,5.0,65.0,0.0,7.0,0,309.0,3.0
2,0,2024-01-01 12:00:00,2024,1,1,12,0,POINT (-73.9383 40.7923272),0.0,Monday,...,0.0,0.0,0.378298,4.0,82.0,0.0,6.0,0,309.0,3.0
3,0,2024-01-01 14:00:00,2024,1,1,14,0,POINT (-73.9383 40.7923272),0.0,Monday,...,0.0,0.0,0.655962,5.0,73.0,0.0,6.0,1,309.0,3.0
4,0,2024-01-01 16:00:00,2024,1,1,16,0,POINT (-73.9383 40.7923272),0.0,Monday,...,0.0,0.0,0.661161,7.0,60.0,0.0,6.0,1,309.0,3.0


In [11]:
# Point geometries are saved as str
type(selected_rentals.coordinates[0])

str

In [12]:
# Converting to Point objects
from shapely import Point

coord_list = []

for i in selected_rentals.coordinates:
    list_conv = i[7:-1].split()
    lng = list_conv[0]
    lat = (list_conv[1])
    coord_list.append(Point(lng, lat))

coord_list[:5] # converted to Point geometries

[<POINT (-73.938 40.792)>,
 <POINT (-73.938 40.792)>,
 <POINT (-73.938 40.792)>,
 <POINT (-73.938 40.792)>,
 <POINT (-73.938 40.792)>]

In [13]:
# Verifying
selected_rentals["coordinates"] = coord_list
type(selected_rentals.coordinates[0])

shapely.geometry.point.Point

## Creating a Citibike station df
This is done because each station is repeated many time in the rentals dataframe. Going to each of the stations repeatedly will take too much time.

In [14]:
import numpy as np

# Defining the df
citibike_stations = pd.DataFrame()
citibike_stations["Citibike_st_id"] = selected_rentals.ID.unique()
citibike_stations.location = Point(0,0)

# Adding location info
for i in range(len(citibike_stations)):
    citibike_stations.loc[i,"location"] = selected_rentals.loc[selected_rentals.ID == citibike_stations.Citibike_st_id[i],"coordinates"].iloc[0]

In [15]:
# Preview
citibike_stations.head()

,Citibike_st_id,location
0,0,POINT (-73.9383 40.7923272)
1,9,POINT (-73.94594 40.7817212)
2,54,POINT (-73.92743647098541 40.772768286288304)
3,55,POINT (-73.9225403 40.7774552)
4,62,POINT (-73.9349 40.8006721)


In [16]:
# Saved as a Point geometry
type(citibike_stations.loc[0,"location"])

shapely.geometry.point.Point

## Generating Buffer Zones
It is mentioned that 74% of Citibike stations are within 400m of any subway entrance in NYC. Keeping some margin of error, if a <u>buffer zone of **800m** is created</u> for each Citibike station, the number of subway stations lying within this proximity can be calculated. An extended buffer is considered because Upper Manhattan area as well as parts of Bronx and Queens considered have lense dense coverage for Citibike stations, and subway stops are farther apart compared to busy/high-demand areas.
<br><br>
[Reference: Pg 9](https://wagner.nyu.edu/files/faculty/publications/Citi_Bike_First_Two_Years_RudinCenter.pdf?utm_source=chatgpt.com)

In [17]:
# creating geodf for citibike stations

import geopandas as gpd
citibike_stations_gdf = gpd.GeoDataFrame(citibike_stations, geometry="location", crs=4326) 
citibike_stations_gdf.head()

,Citibike_st_id,location
0,0,POINT (-73.9383 40.79233)
1,9,POINT (-73.94594 40.78172)
2,54,POINT (-73.92744 40.77277)
3,55,POINT (-73.92254 40.77746)
4,62,POINT (-73.9349 40.80067)


In [18]:
# creating a buffer zone of 500m
citibike_stations_gdf = citibike_stations_gdf.to_crs(epsg=32633)
citibike_stations_gdf["buffer_zone"] = citibike_stations_gdf.geometry.buffer(800)
citibike_stations_gdf = citibike_stations_gdf.to_crs(epsg=4326)
citibike_stations_gdf.head()

# Note: buffer zones must be calculated by reprojecting to epsg 32633
#       for calculating distance in metres

,Citibike_st_id,location,buffer_zone
0,0,POINT (-73.9383 40.79233),"POLYGON ((-5807759.242 9861244.822, -5807763.0..."
1,9,POINT (-73.94594 40.78172),"POLYGON ((-5809583.424 9862177.916, -5809587.2..."
2,54,POINT (-73.92744 40.77277),"POLYGON ((-5811032.839 9859751.982, -5811036.6..."
3,55,POINT (-73.92254 40.77746),"POLYGON ((-5810220.739 9859144.07, -5810224.59..."
4,62,POINT (-73.9349 40.80067),"POLYGON ((-5806333.63 9860847.179, -5806337.48..."


In [19]:
# Visualising the zones

import folium

# Sample dataset with coordinates for Citibike stations
data = {
    'Station': list((citibike_stations_gdf.Citibike_st_id)),
    'Latitude': [x.y for x in citibike_stations_gdf.location],
    'Longitude': [x.x for x in citibike_stations_gdf.location]
}
df = pd.DataFrame(data)

# Create a folium map centered around NYC
m = folium.Map(location=[40.8052, -73.939], zoom_start=14)

# Add station markers
for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']], 
        icon=folium.Icon(color="darkblue", icon="info-sign")
    ).add_to(m)

# Adding geometries for the buffer zones
folium.GeoJson(gpd.GeoSeries(citibike_stations_gdf.buffer_zone).to_crs(4326), name="Buffer Zones").add_to(m)

# Show map
m

## Calculating Subway Stations within Buffer Zones
The number of subway stations that are contained within the buffer zone for each station are calculated here. According to Wang & Noland (2021), subway station lying in proximity to bike-sharing stations/docks has positive correlation with rentals made. In other words, having subway stations around bike-sharing docks is likely to increase rentals.
<br><br>
[Reference](https://pmc.ncbi.nlm.nih.gov/articles/PMC8711868/?utm_source=chatgpt.com)

In [20]:
# Visualising the zones with subway stations

# Sample dataset with coordinates for subway stations
data = {
    'Station': list((stations_gdf["Station IDs"])),
    'Latitude': [x.y for x in stations_gdf.coordinates],
    'Longitude': [x.x for x in stations_gdf.coordinates]
}
df = pd.DataFrame(data)

# Create a folium map centered around NYC
m2 = folium.Map(location=[40.8052, -73.939], zoom_start=13)

# Add station markers
for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']], 
        icon=folium.Icon(color="black", icon="info-sign")
    ).add_to(m2)

# Adding geometries for the buffer zones
folium.GeoJson(gpd.GeoSeries(citibike_stations_gdf.buffer_zone).to_crs(4326), name="Buffer Zones").add_to(m2)

# Show map
m2

So we see that there can in fact be zones that have zero subway stations within a vicinity of 500m.

In [21]:
# To check if coordinates are lying within a polygon, coordinates must be re-projected
polygon = citibike_stations_gdf.buffer_zone[0]
gdf_polygon = gpd.GeoDataFrame(geometry=[polygon], crs="EPSG:32633")
gdf_point = gpd.GeoDataFrame(geometry=[citibike_stations_gdf.location[0]], crs="EPSG:4326")
gdf_point = gdf_point.to_crs(32633)
gdf_polygon.covers(gdf_point.geometry.iloc[0])

0    True
dtype: bool

In [22]:
# Generating a buffer only gdf; crs altered
buffer_gdf = gpd.GeoDataFrame(citibike_stations_gdf[["Citibike_st_id","buffer_zone"]], geometry="buffer_zone", crs="EPSG:32633")
buffer_gdf.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [23]:
# creating a subway stations gdf with altered crs
subway_points_gdf = gpd.GeoDataFrame(stations_gdf[["Station IDs", "coordinates"]],geometry="coordinates",crs="EPSG:4326")
subway_points_gdf = subway_points_gdf.to_crs(32633)
subway_points_gdf.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [24]:
# Performing a left spatial join: only point geom retained

# The resulting df contains NaN values for Citibike_st_id
result = gpd.sjoin(subway_points_gdf, buffer_gdf, how="left", predicate="intersects")

print(f"Number of unique subway stations that lie within the buffer zones: {len(result['Station IDs'].unique())- len(result.loc[np.isnan(result.Citibike_st_id),'Station IDs'].unique())}")
print(" ")
print(f"Number of Citibike stations that have at least one subway station within their buffer zone: {len(result.Citibike_st_id.unique()[~np.isnan(result.Citibike_st_id.unique())])}")
result.head()

Number of unique subway stations that lie within the buffer zones: 40
 
Number of Citibike stations that have at least one subway station within their buffer zone: 151


,Station IDs,coordinates,index_right,Citibike_st_id
0,1,POINT (-5811391.64 9857779.16),71.0,823.0
0,1,POINT (-5811391.64 9857779.16),15.0,85.0
0,1,POINT (-5811391.64 9857779.16),18.0,92.0
1,2,POINT (-5812222.765 9858504.037),18.0,92.0
1,2,POINT (-5812222.765 9858504.037),31.0,156.0


In [25]:
# Adding the number of subway stations for each Citibike station (in buffer zone)
bike_sts_to_consider = list(result.Citibike_st_id.unique()[~np.isnan(result.Citibike_st_id.unique())])

for i in bike_sts_to_consider:
    citibike_stations_gdf.loc[citibike_stations_gdf.Citibike_st_id==i,"sb_st_count"] = len(result[result.Citibike_st_id==i])
    
citibike_stations_gdf.head()

,Citibike_st_id,location,buffer_zone,sb_st_count
0,0,POINT (-73.9383 40.79233),"POLYGON ((-5807759.242 9861244.822, -5807763.0...",NaN
1,9,POINT (-73.94594 40.78172),"POLYGON ((-5809583.424 9862177.916, -5809587.2...",1.0
2,54,POINT (-73.92744 40.77277),"POLYGON ((-5811032.839 9859751.982, -5811036.6...",NaN
3,55,POINT (-73.92254 40.77746),"POLYGON ((-5810220.739 9859144.07, -5810224.59...",NaN
4,62,POINT (-73.9349 40.80067),"POLYGON ((-5806333.63 9860847.179, -5806337.48...",1.0


In [26]:
# Replacing nan values with 0; bike stations with no subway sts in proximity
citibike_stations_gdf["sb_st_count"] = citibike_stations_gdf["sb_st_count"].fillna(0)
citibike_stations_gdf["sb_st_count"].unique()

array([0., 1., 2., 3., 4.])

In [27]:
# How subway stations are represented in buffer zones of various bike stations
print(f'Bike stations with no subway in proximity: {len(citibike_stations_gdf[citibike_stations_gdf["sb_st_count"]==0])}')
print(f'Bike stations with only one subway st. in proximity: {len(citibike_stations_gdf[citibike_stations_gdf["sb_st_count"]==1])}')
print(f'Bike stations with two subway sts. in proximity: {len(citibike_stations_gdf[citibike_stations_gdf["sb_st_count"]==2])}')
print(f'Bike stations with three subway sts. in proximity: {len(citibike_stations_gdf[citibike_stations_gdf["sb_st_count"]==3])}')
print(f'Bike stations with four subway sts. in proximity: {len(citibike_stations_gdf[citibike_stations_gdf["sb_st_count"]==4])}')

Bike stations with no subway in proximity: 49
Bike stations with only one subway st. in proximity: 68
Bike stations with two subway sts. in proximity: 77
Bike stations with three subway sts. in proximity: 5
Bike stations with four subway sts. in proximity: 1


In [28]:
# Visualising the 151/200 important bike station buffers (blue circles), containing at least one subway station (black pin)

# Changing the crs for the "result" df
result = result.to_crs(4326)

data = {
    'Station': list(result.loc[~(np.isnan(result.Citibike_st_id)),"Citibike_st_id"]),
    'Latitude': [x.y for x in list(result.loc[~(np.isnan(result.Citibike_st_id)),"coordinates"])],
    'Longitude': [x.x for x in list(result.loc[~(np.isnan(result.Citibike_st_id)),"coordinates"])]
}
df = pd.DataFrame(data)

# Create a folium map centered around NYC
m4 = folium.Map(location=[40.8052, -73.939], zoom_start=13)

# Add station markers
for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']], 
        icon=folium.Icon(color="black", icon="info-sign")
    ).add_to(m4)

# Adding geometries for the buffer zones
folium.GeoJson(gpd.GeoSeries(citibike_stations_gdf[citibike_stations_gdf.Citibike_st_id.isin(bike_sts_to_consider)].buffer_zone).to_crs(4326), name="Selected Buffer Zones").add_to(m4)

# Show map
m4

## Adding the Information to Rentals Dataset


In [29]:
# Looping through all stations
for i in list(citibike_stations_gdf.Citibike_st_id):
    selected_rentals.loc[selected_rentals.ID == i, "sb_st_800"] = citibike_stations_gdf.loc[citibike_stations_gdf.Citibike_st_id==i,"sb_st_count"].iloc[0]

In [30]:
# Verifying
selected_rentals.sb_st_800.unique()

array([0., 1., 2., 3., 4.])

### Is there any performance increase?
The performance is compared with the baseline level, that was evaluated in the notebook *Solving the problem with lag variable*. Since the variable has only 3 categories, it can be used as a dummy.

In [31]:
# creating dummies for ID and sb_st_800
selected_rentals_dum = pd.get_dummies(selected_rentals[["#_rentals", "year", "month", "day", "hour", "ID", "#_rentals_lag_1", "sb_st_800"]], columns = ["ID", "sb_st_800"], drop_first=False)
selected_rentals_dum.head()

# Developing a training/test data for Feb

# training data
X_train = selected_rentals_dum.loc[(selected_rentals_dum["day"] <=25) & (selected_rentals_dum["month"] == 2),list(selected_rentals_dum.columns)[1:]]
y_train = selected_rentals_dum.loc[(selected_rentals_dum["day"] <=25) & (selected_rentals_dum["month"] == 2), "#_rentals"]

# test data
X_test = selected_rentals_dum.loc[(selected_rentals_dum["day"] >25) & (selected_rentals_dum["month"] == 2),list(selected_rentals_dum.columns)[1:]]
y_test = selected_rentals_dum.loc[(selected_rentals_dum["day"] >25) & (selected_rentals_dum["month"] == 2), "#_rentals"]

# training the model
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=2) 
forest.fit(X_train, y_train)

# testing performance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
y_test_pred = forest.predict(X_test)
print(mean_squared_error(y_test, y_test_pred),r2_score(y_test, y_test_pred))

# Developing a training/test data for Mar

# training data
X_train = selected_rentals_dum.loc[(selected_rentals_dum["day"] <=25) & (selected_rentals_dum["month"] == 3),list(selected_rentals_dum.columns)[1:]]
y_train = selected_rentals_dum.loc[(selected_rentals_dum["day"] <=25) & (selected_rentals_dum["month"] == 3), "#_rentals"]

# test data
X_test = selected_rentals_dum.loc[(selected_rentals_dum["day"] >25) & (selected_rentals_dum["month"] == 3),list(selected_rentals_dum.columns)[1:]]
y_test = selected_rentals_dum.loc[(selected_rentals_dum["day"] >25) & (selected_rentals_dum["month"] == 3), "#_rentals"]

# training the model
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=2) 
forest.fit(X_train, y_train)

# testing performance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
y_test_pred = forest.predict(X_test)
print(mean_squared_error(y_test, y_test_pred),r2_score(y_test, y_test_pred))

2.6885593749999996 0.26607938722451563
3.8903557857142856 0.44840827572419806


There is noticeable increase in performance, particularly for the month of Feb.

In [32]:
# creating dummies for ID and sb_st_800
selected_rentals_dum = pd.get_dummies(selected_rentals[["#_rentals", "year", "month", "day", "hour", "ID", "#_rentals_lag_1", "sb_st_800"]], columns = ["ID"], drop_first=False)
selected_rentals_dum.head()

# Developing a training/test data for Feb

# training data
X_train = selected_rentals_dum.loc[(selected_rentals_dum["day"] <=25) & (selected_rentals_dum["month"] == 2),list(selected_rentals_dum.columns)[1:]]
y_train = selected_rentals_dum.loc[(selected_rentals_dum["day"] <=25) & (selected_rentals_dum["month"] == 2), "#_rentals"]

# test data
X_test = selected_rentals_dum.loc[(selected_rentals_dum["day"] >25) & (selected_rentals_dum["month"] == 2),list(selected_rentals_dum.columns)[1:]]
y_test = selected_rentals_dum.loc[(selected_rentals_dum["day"] >25) & (selected_rentals_dum["month"] == 2), "#_rentals"]

# training the model
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=2) 
forest.fit(X_train, y_train)

# testing performance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
y_test_pred = forest.predict(X_test)
print(mean_squared_error(y_test, y_test_pred),r2_score(y_test, y_test_pred))

# Developing a training/test data for Mar

# training data
X_train = selected_rentals_dum.loc[(selected_rentals_dum["day"] <=25) & (selected_rentals_dum["month"] == 3),list(selected_rentals_dum.columns)[1:]]
y_train = selected_rentals_dum.loc[(selected_rentals_dum["day"] <=25) & (selected_rentals_dum["month"] == 3), "#_rentals"]

# test data
X_test = selected_rentals_dum.loc[(selected_rentals_dum["day"] >25) & (selected_rentals_dum["month"] == 3),list(selected_rentals_dum.columns)[1:]]
y_test = selected_rentals_dum.loc[(selected_rentals_dum["day"] >25) & (selected_rentals_dum["month"] == 3), "#_rentals"]

# training the model
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(random_state=2) 
forest.fit(X_train, y_train)

# testing performance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
y_test_pred = forest.predict(X_test)
print(mean_squared_error(y_test, y_test_pred),r2_score(y_test, y_test_pred))

2.680677267857143 0.26823103801491255
3.881087821428572 0.4497223283925056


Using the **sb_st_800** not as a dummy is actually resulting in better performance! So, it should be used as a regular variable.

In [35]:
# Exporting the dataset
selected_rentals.to_csv('rentals_working_with_subway_stations.csv', index=False)